In [9]:
import numpy as np
import copy
import matplotlib.pyplot as plt
import random as rd
import math as math
from matplotlib import colors
import pandas as pd
import statistics as statistics

data = np.empty((27000, 13))
scenario = 0 #this variable is for tracking which combination of variables the habitat is

thermoregulating = 1

def map(A):
     
    arrangment = A
    hotmean = 42
    warmH = 2.25
    coolH = 1.125
    
    # shade arrangment: 1, 4, or 16 patches
    # %19.14 (784)
            #1: 28x28| 4: 14x14| 16: 7x7
    
    tempwarm = np.random.normal(hotmean, warmH, 4096)
    tempcool = np.random.normal(34, coolH, 4096)
    tempwarm = tempwarm.reshape(64,64)
    tempcool = tempcool.reshape(64,64)
    
    temp = tempwarm
    if arrangment == 1:
        size = 28
        for i in range (size):
            for j in range (size):
                temp[i][j]=tempcool[i][j]
            
    elif arrangment == 4:
        size = 14
        for i in range (size):
            for j in range (size):
                temp[(16-int(size/2))+i][(16-int(size/2))+j]=tempcool[(16-int(size/2))+i][(16-int(size/2))+j]
                temp[(16-int(size/2))+i][(48-int(size/2))+j]=tempcool[(16-int(size/2))+i][(48-int(size/2))+j]
                temp[(48-int(size/2))+i][(16-int(size/2))+j]=tempcool[(48-int(size/2))+i][(16-int(size/2))+j]
                temp[(48-int(size/2))+i][(48-int(size/2))+j]=tempcool[(48-int(size/2))+i][(48-int(size/2))+j]
            
    else:
        size = 7
        for i in range (size):
            for j in range (size):
                temp[(8-int(size/2))+i][(8-int(size/2))+j]=tempcool[(8-int(size/2))+i][(8-int(size/2))+j]
                temp[(8-int(size/2))+i][(24-int(size/2))+j]=tempcool[(8-int(size/2))+i][(24-int(size/2))+j]
                temp[(8-int(size/2))+i][(40-int(size/2))+j]=tempcool[(8-int(size/2))+i][(40-int(size/2))+j]
                temp[(8-int(size/2))+i][(56-int(size/2))+j]=tempcool[(8-int(size/2))+i][(56-int(size/2))+j]
                    
                temp[(24-int(size/2))+i][(8-int(size/2))+j]=tempcool[(24-int(size/2))+i][(8-int(size/2))+j]
                temp[(24-int(size/2))+i][(24-int(size/2))+j]=tempcool[(24-int(size/2))+i][(24-int(size/2))+j]
                temp[(24-int(size/2))+i][(40-int(size/2))+j]=tempcool[(24-int(size/2))+i][(40-int(size/2))+j]
                temp[(24-int(size/2))+i][(56-int(size/2))+j]=tempcool[(24-int(size/2))+i][(56-int(size/2))+j]
    
                temp[(40-int(size/2))+i][(8-int(size/2))+j]=tempcool[(40-int(size/2))+i][(8-int(size/2))+j]
                temp[(40-int(size/2))+i][(24-int(size/2))+j]=tempcool[(40-int(size/2))+i][(24-int(size/2))+j]
                temp[(40-int(size/2))+i][(40-int(size/2))+j]=tempcool[(40-int(size/2))+i][(40-int(size/2))+j]
                temp[(40-int(size/2))+i][(56-int(size/2))+j]=tempcool[(40-int(size/2))+i][(56-int(size/2))+j]
                    
                temp[(56-int(size/2))+i][(8-int(size/2))+j]=tempcool[(56-int(size/2))+i][(8-int(size/2))+j]
                temp[(56-int(size/2))+i][(24-int(size/2))+j]=tempcool[(56-int(size/2))+i][(24-int(size/2))+j]
                temp[(56-int(size/2))+i][(40-int(size/2))+j]=tempcool[(56-int(size/2))+i][(40-int(size/2))+j]
                temp[(56-int(size/2))+i][(56-int(size/2))+j]=tempcool[(56-int(size/2))+i][(56-int(size/2))+j]
    
    return temp



class Individual:
    def __init__(self,name,mass,tlow,thigh):
        self.name = name
        self.mass = mass #mass (g)
        self.loc_x = np.random.uniform(63,1)
        self.loc_y = np.random.uniform(63,1)
        #self.Tb = temp[math.floor(self.loc_x-1),math.floor(self.loc_y-1)]
        #self.Te = temp[math.floor(self.loc_x-1),math.floor(self.loc_y-1)]
        self.Tb = 34
        self.Te = 34
        self.tlow = tlow
        self.thigh = thigh
        self.Te_history = []
        self.Tb_history = []
        self.energy = 0
        self.distance = 0
        self.interaction_count = 0
        self.coordinates_x = [self.loc_x]
        self.coordinates_y = [self.loc_y]
        self.orientation = np.random.uniform(math.radians(359) ,math.radians(0))
        self.MU=0.
        self.KAPPA=1.
        self.BETA=4.
        self.ALPHA=1.
        self.DMAX=8*(math.log10(self.mass)+1)
        self.decision=6
        self.chance_to_move=0.25
        self.is_competing = False #individuals can only compete once. if near an individuals get switched to true for that time step
        
    def move(self):
        choice_distance_inrange=[]
        choice_temp_inrange=[]
        choice_x_inrange=[]
        choice_y_inrange=[]
        
        choice_distance_outrange=[]
        choice_temp_outrange=[]
        choice_x_outrange=[]
        choice_y_outrange=[]
        
        pref_available=0
        distance = 0
        
        if thermoregulating == 1 and (self.Tb < self.tlow or self.Tb > self.thigh):
            for d in range(self.decision):
                
                if self.loc_x > 63 or self.loc_x < 1 or self.loc_y > 63 or self.loc_y < 1:
                    angle = np.random.vonmises(math.radians(180),8.0) + self.orientation
                else:
                    angle = (np.random.vonmises(self.MU, self.KAPPA)) + self.orientation
                distance = (np.random.beta(self.ALPHA,self.BETA)) * self.DMAX
                x = (math.cos(angle)*distance) + self.loc_x
                y = (math.sin(angle)*distance) + self.loc_y
                if x >64:
                    x = 64
                if x<0:
                    x= 0
                if y >64:
                    y = 64
                if y<0:
                    y= 0
                Te = temp[math.floor(x-1),math.floor(y-1)]
                if Te >= self.Tb:
                    tau = math.exp(0.72+0.36*math.log(self.mass))
                    Tb = (math.exp(-1./tau) * (self.Tb - Te) + Te)
                else:
                    tau = math.exp(0.42+0.44*math.log(self.mass))
                    Tb = (math.exp(-1./tau) * (self.Tb - Te) + Te)
   
                if self.tlow <= Tb and self.thigh>=Tb:        
                    pref_available=1
                    choice_x_inrange.append(x)
                    choice_y_inrange.append(y)
                    choice_temp_inrange.append(Tb)
                    choice_distance_inrange.append(distance)
                else:
                    choice_x_outrange.append(x)
                    choice_y_outrange.append(y)
                    choice_temp_outrange.append(Tb)
                    choice_distance_outrange.append(distance)

            if pref_available == 1:
                #select the closest pref temp
                selection = choice_distance_inrange.index(min(choice_distance_inrange))
                move_x = choice_x_inrange[selection]
                move_y = choice_y_inrange[selection]
            else:
                if self.Tb < self.tlow:
                    #select the closest warmer temp
                    if max(choice_temp_outrange) >= self.Tb:
                        selection = choice_temp_outrange.index(max(choice_temp_outrange))
                    else:
                        selection = np.random.randint(0,self.decision-1)
                    move_x = choice_x_outrange[selection]
                    move_y = choice_y_outrange[selection]
                else:
                    #select the closest cooler temp
                    if min(choice_temp_outrange) <= self.Tb:
                        selection = choice_temp_outrange.index(min(choice_temp_outrange))
                    else:
                        selection = np.random.randint(0,self.decision-1)
                    move_x = choice_x_outrange[selection]
                    move_y = choice_y_outrange[selection]
        else:
            if np.random.uniform(1,0) <= self.chance_to_move:
                if self.loc_x > 63 or self.loc_x < 1 or self.loc_y > 63 or self.loc_y < 1:
                    angle = np.random.vonmises(math.radians(180),8.0) + self.orientation
                else:
                    angle = (np.random.vonmises(self.MU, self.KAPPA)) + self.orientation
                distance = (np.random.beta(self.ALPHA,self.BETA)) * self.DMAX
                move_x = math.cos(self.orientation)*distance + self.loc_x
                move_y = math.sin(self.orientation)*distance + self.loc_y
            else:
                move_x = self.loc_x
                move_y = self.loc_y
                angle = self.orientation
        if move_x >64:
            move_x = 64
        if move_x<0:
            move_x= 0
        if move_y >64:
            move_y = 64
        if move_y<0:
            move_y= 0
        self.loc_x = move_x
        self.loc_y = move_y
        self.orientation = angle
        self.coordinates_x.append(self.loc_x)
        self.coordinates_y.append(self.loc_y)
        self.energy_analysis(distance)
        return None

    
    def operative_temperature(self):
        #self.Te = (self.loc_y+16)-(self.loc_y*0.54)
        self.Te = temp[math.floor(self.loc_x-1),math.floor(self.loc_y-1)]
        return self.Te

    
    def body_temperature(self,initial_Tb):
        if self.Te >= initial_Tb:
            tau = math.exp(0.72+0.36*math.log(self.mass))
            self.Tb = self.Te + math.exp(-1./tau) * (initial_Tb - self.Te)
            self.Tb_history.append(self.Tb)
        if self.Te < initial_Tb:
            tau = math.exp(0.42+0.44*math.log(self.mass))
            self.Tb = self.Te + math.exp(-1./tau) * (initial_Tb - self.Te) 
            self.Tb_history.append(self.Tb)
        return self.Tb
    
    def energy_analysis(self, moved):
        act_scope = 1. + 4. * moved/self.DMAX
        energy = (act_scope*20.2*10**(0.038*self.Tb-1.771)*self.mass**0.82)/60
        self.energy += energy
        self.distance += moved


def check_proximity(individual_1, individual_2, prox, indiv_id_1, indiv_id_2, pairing_type):   
    #matrix to track all individuals that are within a 2 pixle radius
    if math.dist([individual_1.loc_x,individual_1.loc_y], [individual_2.loc_x,individual_2.loc_y]) <= 2:
        prox.append([math.dist([individual_1.loc_x,individual_1.loc_y], [individual_2.loc_x,individual_2.loc_y]),
                   indiv_id_1, indiv_id_2, pairing_type])
    return prox
        
        
def compete_intra(individuals, prox, cow):
    chance_of_win = cow
    for i in range (len(prox)):
        individual_1 = individuals[prox[i][1]]
        individual_2 = individuals[prox[i][2]]
        if individual_1.is_competing == False and individual_2.is_competing == False:      
            individual_1.interaction_count += 1
            individual_2.interaction_count += 1
            #set that they are competing to ture so they don't compete again this time step
            individual_1.is_competing = True
            individual_2.is_competing = True
            if np.random.uniform(1,0) < chance_of_win:
                loser = individual_2
            else:
                loser = individual_1    
            x = loser.loc_x
            y = loser.loc_y
            angle = np.random.vonmises(math.radians(180),8.0)
            loser.orientation += angle
            distance = (np.random.beta(loser.ALPHA,loser.BETA)) * loser.DMAX
            move_x = math.cos(loser.orientation)*distance
            move_y = math.sin(loser.orientation)*distance
            loser.loc_x += move_x
            loser.loc_y += move_y
            if loser.loc_x >64:
                loser.loc_x = 64
            if loser.loc_x<0:
                loser.loc_x= 0
            if loser.loc_y >64:
                loser.loc_y = 64
            if loser.loc_y<0:
                loser.loc_y= 0
            loser.coordinates_x[-1] = loser.loc_x
            loser.coordinates_y[-1] = loser.loc_y
            loser.energy_analysis(distance)
    
    #after all interactions have occured, reset competing back to false
    for i in range (len(individuals)):
        individuals[i].is_competing = False

    return None

def compete_inter(carol, sag, prox, cow_intra, cow_inter):
    for i in range (len(prox)):
        if prox[i][3] == 1:
            individual_1 = carol[prox[i][1]]
            individual_2 = carol[prox[i][2]]
            chance_of_win = cow_intra
        if prox[i][3] == 2:
            individual_1 = sag[prox[i][1]]
            individual_2 = sag[prox[i][2]]
            chance_of_win = cow_intra
        else:
            individual_1 = sag[prox[i][1]]
            individual_2 = carol[prox[i][2]]
            chance_of_win = cow_inter
            
        if individual_1.is_competing == False and individual_2.is_competing == False:      
            individual_1.interaction_count += 1
            individual_2.interaction_count += 1
            #set that they are competing to ture so they don't compete again this time step
            individual_1.is_competing = True
            individual_2.is_competing = True
            if np.random.uniform(1,0) < chance_of_win:
                loser = individual_2
            else:
                loser = individual_1    
            x = loser.loc_x
            y = loser.loc_y
            angle = np.random.vonmises(math.radians(180),8.0)
            loser.orientation += angle
            distance = (np.random.beta(loser.ALPHA,loser.BETA)) * loser.DMAX
            move_x = math.cos(loser.orientation)*distance
            move_y = math.sin(loser.orientation)*distance
            loser.loc_x += move_x
            loser.loc_y += move_y
            if loser.loc_x >64:
                loser.loc_x = 64
            if loser.loc_x<0:
                loser.loc_x= 0
            if loser.loc_y >64:
                loser.loc_y = 64
            if loser.loc_y<0:
                loser.loc_y= 0
            loser.coordinates_x[-1] = loser.loc_x
            loser.coordinates_y[-1] = loser.loc_y
            loser.energy_analysis(distance)
    
    #after all interactions have occured, reset competing back to false
    for i in range (len(carol)):
        carol[i].is_competing = False
        sag[i].is_competing = False

    return None




for pop in range (0,3): #pop 0 is 160/80, 1 is 80/40, 2 is 40/20        
    for comp in range (0,3): #comp of 0 is for no comp, 1 is intra only and a comp of 2 is for inter and intraspecific competition
        competing = comp
        #slope for chance of winning is temp dependent and based of my empirical results
        #chance_of_win = (0.0175*sag.Tb)+0.2575
        #chance_of_win_inter = 0.6
        #chance_of_win_inter = 0.75
        chance_of_win_inter = 0.9
        chance_of_win_intra = 0.5
        #repetative to have thigh and tlow twice, but its incase I want to change them independently
        carol_tlow = 32
        carol_thigh = 36
        sag_tlow = 32
        sag_thigh = 36
        for r in range (1,4):
            #set the spatial arrangement
            if r == 1:
                a=1
            if r == 2:
                a=4
            if r == 3:
                a=16
            for t in range (1000):
                temp = map(a)
                sag_Tb = []
                sag_energy = []
                sag_distance = []
                sag_interaction_count = []
                carol_Tb = []
                carol_energy = []
                carol_distance = []
                carol_interaction_count = []
                if pop == 0:
                    if comp != 2:
                        count = 200
                    else:
                        count =100
                elif pop == 1:
                    if comp != 2:
                        count = 100
                    else:
                        count =50
                else:
                    if comp != 2:
                        count = 50
                    else:
                        count =25
                #else:
                    #if comp != 2:
                        #count = 20
                    #else:
                        #count = 10                     

                carol = []
                for i in range (count):
                    carol.append(Individual('carol',10,carol_tlow,carol_thigh))
                if competing == 2: #only create sag if inter competition is occuring
                    sag = []
                    for i in range (count):
                        sag.append(Individual('sag',10,sag_tlow,sag_thigh))

                for i in range(15):
                    carol_within_range = []
                    sag_within_range = []
                    #inter list will have the other two groups appended to it after all the distant checks are made
                    #this is so there's one list with all distances that is then sorted from closest to farthest
                    inter_within_range = []
                    competing_individuals = []
                    
                    if competing == 0: #no competition
                        for j in range (count):
                            carol[j].operative_temperature()
                            carol[j].body_temperature(carol[j].Tb)
                            carol[j].move()
                    elif competing == 1: #intra competition only
                        for j in range (count):
                            carol[j].operative_temperature()
                            carol[j].body_temperature(carol[j].Tb)
                            carol[j].move()
                        for j in range (count):
                            for y in range (count):
                                if j < y: #don't want to compare to individuals that have already been compared to or self
                                    #intra competition
                                    #last variable is pairing type. not important for intra only, 
                                    #but important when tracking two intra types and inter
                                    check_proximity(carol[y], carol[j], carol_within_range, y, j, 1)
                        carol_within_range.sort()
                        compete_intra(carol, carol_within_range, chance_of_win_intra)
                    else: #inter and intra competition
                        for y in range (count):
                            sag[y].operative_temperature()
                            carol[y].operative_temperature()
                            sag[y].body_temperature(sag[y].Tb)   
                            carol[y].body_temperature(carol[y].Tb)
                            sag[y].move()
                            carol[y].move()
                        for j in range (count):
                            for y in range (count):
                                if j < y:
                                    #intra competition
                                    check_proximity(sag[y], sag[j], sag_within_range, y, j, 2)
                                    check_proximity(carol[y], carol[j], carol_within_range, y, j, 1)
                                #inter competition
                                check_proximity(sag[y], carol[j], inter_within_range, y, j, 3)
                        #append all groups together and sort
                        for p in range (len(carol_within_range)):
                            inter_within_range.append(carol_within_range[p])
                        for p in range (len(sag_within_range)):
                            inter_within_range.append(sag_within_range[p])
                        inter_within_range.sort()
                        compete_inter(carol, sag, inter_within_range, chance_of_win_intra, chance_of_win_inter)

                for i in range (count):
                    carol_Tb.append(carol[i].Tb_history)
                    carol_energy.append(carol[i].energy)
                    carol_distance.append(carol[i].distance)
                    carol_interaction_count.append(carol[i].interaction_count)
                    if competing == 2:
                        sag_Tb.append(sag[i].Tb_history)
                        sag_energy.append(sag[i].energy)
                        sag_distance.append(sag[i].distance)
                        sag_interaction_count.append(sag[i].interaction_count)        
                
                carol_tb_temp=[]
                sag_tb_temp=[]
                carol_ia_temp=[]
                sag_ia_temp=[]
                    
                #calculate thermal innaccuracy
                for p in range (len(carol_Tb)):
                    carolinensis = statistics.median(carol_Tb[p])  
                    carol_tb_temp.append(carolinensis)
                    if competing == 2:
                        sagrei = statistics.median(sag_Tb[p])
                        sag_tb_temp.append(sagrei)
                    if carolinensis < carol_tlow:
                        carolinensis_ia = carol_tlow-carolinensis
                    elif carolinensis > carol_thigh:
                        carolinensis_ia = carolinensis-carol_thigh
                    else:
                        carolinensis_ia = 0
                    if competing == 2:
                        if sagrei < sag_tlow:
                            sagrei_ia = sag_tlow-sagrei
                        elif sagrei > sag_thigh:
                            sagrei_ia = sagrei-sag_thigh
                        else:
                            sagrei_ia = 0
                    carol_ia_temp.append(carolinensis_ia)
                    if competing == 2:
                        sag_ia_temp.append(sagrei_ia)

                #record data
                data[scenario,0] = pop
                data[scenario,1] = comp
                data[scenario,2] = a
                data[scenario,3] = statistics.mean(carol_tb_temp)
                data[scenario,4] = statistics.mean(carol_ia_temp)
                data[scenario,5] = statistics.mean(carol_energy)
                data[scenario,6] = statistics.mean(carol_distance)
            
                if comp == 1:
                    data[scenario,7] = statistics.mean(carol_interaction_count)
                elif comp == 2:
                    data[scenario,7] = statistics.mean(carol_interaction_count)
                    data[scenario,8] = statistics.mean(sag_tb_temp)
                    data[scenario,9] = statistics.mean(sag_ia_temp)
                    data[scenario,10] = statistics.mean(sag_energy)
                    data[scenario,11] = statistics.mean(sag_distance)
                    data[scenario,12] = statistics.mean(sag_interaction_count)

                scenario += 1



In [10]:
import pandas as pd

headerList = ['population_size', 'comp_type', 'shade_arrangment', 'tb','inaccuracy', 
              'energy', 'distance', 
              'interaction_count', 
              'competitor_tb', 'competitor_inaccuracy', 
              'competitor_energy', 
              'competitor_distance', 'competitor_interaction_count'
              ]
data_df = pd.DataFrame(data)
data_df.to_csv('data_90.csv', header=headerList)

In [6]:
inter_within_range

[[0.9700020077019293, 9, 7, 1], [1.7832740562660117, 8, 5, 1]]

In [ ]:
carol_within_range 
sag_within_range
inter_within_range
inter_intra_within_range

In [7]:
carol_within_range

[[1.7832740562660117, 8, 5, 1], [0.9700020077019293, 9, 7, 1]]

In [18]:
for i in range (len(carol_within_range)):
    inter_within_range.append(carol_within_range[i])
inter_within_range.sort()


In [8]:
inter_within_range

[[0.9700020077019293, 9, 7, 1], [1.7832740562660117, 8, 5, 1]]

7